# Answering Business Questions Using SQL

In this project, we will be looking at a database containing information for a fictional digital music shop called Chinook. The database schema are as follows:

![Database Schema](C:\Users\abdi-\OneDrive\Pictures\databaseschema.png)

We will be answering business questions regarding the information in the database, such as:

* what type of genre sells the most?
* which sales agent has the most sales?
* what country has the best sales?
* are albums purchased more, or are individual tracks purchased more?

Our conclusions will give recommendations based on our findings to increase sales, and improve overall business strategy for Chinook.

# Loading & Selecting Database

In [66]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [67]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Genre of Albums to Purchase

We have been given a list of potential artists albums to add to the Chinook record store shown as follows:

![New Artists](C:\Users\abdi-\OneDrive\Pictures\newartists.png)

Based on which genre of music is most popular at Chinook, we will determine which of the new artists albums will be added.

In [68]:
%%sql

WITH usacountry AS
     (
      SELECT *
         FROM customer
      WHERE country LIKE '%USA%'
     ),
     usatotal AS
     (
      SELECT *
         FROM usacountry uc
      INNER JOIN invoice i ON i.customer_id = uc.customer_id
     ),
     usagenre AS
     (
      SELECT g.name genre, il.invoice_id invoice_id
      FROM usatotal ut
      INNER JOIN invoice_line il ON il.invoice_id = ut.invoice_id
      INNER JOIN track t ON t.track_id = il.track_id 
      INNER JOIN genre g ON g.genre_id = t.genre_id
     )   
     

SELECT genre,
       COUNT(invoice_id) tracks_sold, 
       ROUND(CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(*) FROM usagenre),4) percentage
FROM usagenre ug
GROUP BY genre
ORDER BY 2 DESC
LIMIT 20;

Done.


genre,tracks_sold,percentage
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


The most popular genres of music that is sold in the USA based on number of tracks sold are Rock (~ 53.4% of tracks sold), Alternative & Punk (~ 12.4% of tracks sold), and Metal (~ 11.8% of tracks sold). The artists from the record label produce music in the following genres:
    
* Hip-Hop
* Punk
* Pop
* Blues

Of the four genres, Punk (2nd at ~ 12,4%), the Blues (5th at ~ 3.4%), and Pop (8th at ~ 2.1%)are the most popular genres, so therefore we should purchase albums from the artists Red Tone, Slim Jim Bites, and Meteor and the Girls. 

# Amount of Sales ($) per Agent

In [69]:
%%sql

SELECT e.first_name || ' ' || e.last_name employee_name,
       ROUND(SUM(i.total),2) dollar_amount_sales, 
       e.hire_date 
FROM customer c
INNER JOIN employee e ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY support_rep_id
LIMIT 5;

Done.


employee_name,dollar_amount_sales,hire_date
Jane Peacock,1731.51,2017-04-01 00:00:00
Margaret Park,1584.0,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


Given that Jane Peacock, the highest selling agent, was hired the earliest, and Steve Johnson, the lowest selling agent, was hired the most recent, it makes logical sense for the ranking shown.

# Analyzing Sales by Country

In [70]:
%%sql


WITH country_select AS
      (
       SELECT
           CASE WHEN (SELECT COUNT(*) FROM customer WHERE country = c.country) = 1
           THEN 'Other' ELSE c.country
           END AS country,
           c.customer_id,
           i.*
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
      )
    
SELECT country,
       COUNT(DISTINCT(customer_id)) total_customers,
       ROUND(SUM(total),2) total_sales,
       ROUND(SUM(total)/COUNT(DISTINCT(customer_id)),2) average_sales_value,
       ROUND(SUM(total)/COUNT(DISTINCT(invoice_id)),2) average_order_value
FROM country_select
GROUP BY country
ORDER BY CASE WHEN country = 'Other' THEN 1 ELSE 0 END,total_sales DESC



Done.


country,total_customers,total_sales,average_sales_value,average_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


The top 3 countries by total sales are the US, Canada, and Brazil. The Czech Republic seems promising as the average sales value and average order value are the highest out of any country.

# Albums vs Individual Tracks

In [71]:
%%sql

WITH invoicealbums AS
    (
     SELECT t.album_id,
            il.track_id,
            il.invoice_id invoice_id
     FROM invoice_line il
     LEFT JOIN track t ON t.track_id = il.track_id
     GROUP BY 1,2,3
    ),
    
    albums as (
        SELECT  t.album_id albums_id,
                t.track_id tracks_id
        FROM track t
        GROUP BY 1,2
    ),
    
    invoicetracks AS
    (
     SELECT CASE WHEN COUNT(DISTINCT albums_id) = 1 
     AND COUNT(DISTINCT(ia.track_id)) = COUNT(DISTINCT(tracks_id))
     THEN 'album purchase' 
     ELSE 'not album purchase'
     END AS album_purchase,
     invoice_id
     FROM invoicealbums ia
     LEFT JOIN albums a ON a.albums_id = ia.album_id
     GROUP BY ia.invoice_id
    )
    
SELECT album_purchase,
       COUNT(invoice_id) number_of_invoices,
       CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoicetracks) * 100 percent
FROM invoicetracks
GROUP BY album_purchase

Done.


album_purchase,number_of_invoices,percent
album purchase,114,18.566775244299674
not album purchase,500,81.43322475570032


Most of the invoices (500) are individual track purchases, while only 114 invoices are full album purchases. This indicates more attention should be made towards purchasing popular tracks from artists for Chinook rather than full albums.

# Top Artist

In [72]:
%%sql

WITH track_num AS
    (
     SELECT ar.name name, 
            g.name genre,
            * 
     FROM track t
     INNER JOIN playlist_track p ON p.track_id = t.track_id
     INNER JOIN album a ON a.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = a.artist_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
    )

SELECT 
    name artist,
    COUNT(name) frequency,
    genre 
FROM track_num
GROUP BY name
ORDER BY frequency DESC
LIMIT 10

Done.


artist,frequency,genre
Iron Maiden,516,Metal
U2,333,Pop
Metallica,296,Metal
Led Zeppelin,252,Rock
Deep Purple,226,Rock
Lost,184,Drama
Pearl Jam,177,Rock
Eric Clapton,145,Latin
Faith No More,145,Alternative & Punk
Lenny Kravitz,143,Metal


With the top 10 artists by frequency throughout all 18 playlists shown, we can see that Iron Maiden, U2, and Metallica are the 3 most popular artists. This is a good reference point for determining which artists tracks/albums to purchase for Chinook.

# Purchased vs Not Purchased

In [73]:
%%sql

SELECT
    CASE WHEN il.track_id = t.track_id THEN 'yes' 
    ELSE 'no'
    END AS track_purchased,
    COUNT(t.track_id) num_tracks
FROM track t
LEFT JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY track_purchased



Done.


track_purchased,num_tracks
no,1697
yes,4757


4757 tracks have been purchase while only 1697 tracks have not. We will explore the tracks that have not been purchased and compare them to see the differences in popularity. 

# Non-purchased Tracks

In [74]:
%%sql

WITH notracks AS
    (
     SELECT
     CASE WHEN il.track_id = t.track_id THEN 'yes' 
     ELSE 'no'
     END AS track_purchased,
     *
FROM track t
LEFT JOIN invoice_line il ON il.track_id = t.track_id
    )

SELECT
       COUNT(DISTINCT nt.name) num_tracks, 
       a.title album, 
       ar.name artist,
       p.name playlist
       FROM notracks nt
INNER JOIN album a ON a.album_id = nt.album_id
INNER JOIN artist ar ON ar.artist_id = a.artist_id
INNER JOIN playlist_track pt ON pt.track_id = nt.track_id
INNER JOIN playlist p ON p.playlist_id = pt.playlist_id
INNER JOIN genre g ON g.genre_id = nt.genre_id
WHERE track_purchased = 'no'
GROUP BY playlist,album
ORDER BY num_tracks DESC
LIMIT 10; 

 

Done.


num_tracks,album,artist,playlist
33,Greatest Hits,Lenny Kravitz,Music
25,Minha Historia,Chico Buarque,Music
25,"Lost, Season 3",Lost,TV Shows
24,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),TV Shows
24,"Lost, Season 1",Lost,TV Shows
24,"Lost, Season 2",Lost,TV Shows
24,"The Office, Season 3",The Office,TV Shows
22,Up An' Atom,Gene Krupa,Music
22,"Heroes, Season 1",Heroes,TV Shows
22,"The Office, Season 2",The Office,TV Shows


In [75]:
%%sql


WITH notracks AS
    (
     SELECT
     CASE WHEN il.track_id = t.track_id THEN 'yes' 
     ELSE 'no'
     END AS track_purchased,
     *
FROM track t
LEFT JOIN invoice_line il ON il.track_id = t.track_id
),

    playlist_freq AS

(
    SELECT
       g.name genre,
       a.album_id,COUNT(DISTINCT nt.name) num_tracks, 
       a.title album, 
       ar.name artist,
       p.name playlist 
       FROM notracks nt
INNER JOIN album a ON a.album_id = nt.album_id
INNER JOIN artist ar ON ar.artist_id = a.artist_id
INNER JOIN playlist_track pt ON pt.track_id = nt.track_id
INNER JOIN playlist p ON p.playlist_id = pt.playlist_id
INNER JOIN genre g ON g.genre_id = nt.genre_id
WHERE track_purchased = 'no'
GROUP BY playlist,album
ORDER BY num_tracks DESC
LIMIT 10

)

SELECT playlist,
       SUM(num_tracks) total_tracks,
       ROUND(CAST(SUM(num_tracks) AS FLOAT)/
       (SELECT SUM(num_tracks) FROM playlist_freq),2) percent_of_total
FROM playlist_freq
GROUP BY playlist
ORDER BY total_tracks DESC



Done.


playlist,total_tracks,percent_of_total
TV Shows,165,0.67
Music,80,0.33


Of the top 10 non-purchased tracks based on their frequency in appearing on playlists, 7 are from the TV Show playlist category (including well known TV series such as Lost, The Office, etc.). As a percent of the total number of unpurchased tracks in the top 10, TV Shows make up about 67% of the total, which indicates that the decision making with regards to purchasing tracks belonging to the TV Show category for Chinook should be re-evaluated.

# Track Sales & Popularity

In [76]:
%%sql

WITH track_select AS
    (
     SELECT il.track_id track_id,
            *  
            FROM invoice i
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     INNER JOIN track t ON t.track_id = il.track_id
    )

SELECT 
    COUNT(ts.track_id) track_frequency,
    ts.name track, 
    a.title album, 
    ar.name artist,
    ROUND(SUM(ts.total),2) 
    total_sales FROM track_select ts
INNER JOIN album a ON a.album_id = ts.album_id
INNER JOIN artist ar ON ar.artist_id = a.artist_id
GROUP BY ts.name
ORDER BY total_sales DESC
LIMIT 10

Done.


track_frequency,track,album,artist,total_sales
33,War Pigs,Cake: B-Sides and Rarities,Cake,246.51
13,Hey Joe,Are You Experienced?,Jimi Hendrix,177.21
14,Are You Experienced?,Are You Experienced?,Jimi Hendrix,175.23
13,Third Stone From The Sun,Are You Experienced?,Jimi Hendrix,171.27
14,Highway Chile,Are You Experienced?,Jimi Hendrix,169.29
11,Foxy Lady,Are You Experienced?,Jimi Hendrix,164.34
11,Remember,Are You Experienced?,Jimi Hendrix,158.4
11,May This Be Love,Are You Experienced?,Jimi Hendrix,157.41
12,Love Or Confusion,Are You Experienced?,Jimi Hendrix,150.48
10,Purple Haze,Are You Experienced?,Jimi Hendrix,148.5


From looking at the tracks that have sold the most, we can see that Jimi Hendrix's album 'Are You Experienced?' has 9 of the top 10 tracks sold at Chinook.

# Protected vs Non-protected Media Types

In [77]:
%%sql

WITH mediatypesales AS 
    (
     SELECT *,mt.name mediatype FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN invoice_line il ON il.track_id = t.track_id
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    )

SELECT 
mediatype,
ROUND(SUM(mts.total),2) total_sales,
ROUND((SUM(mts.total))/
      (SELECT SUM(mts.total) 
       FROM mediatypesales mts)*100,2) 
       percent_of_sales
FROM mediatypesales mts
GROUP BY mediatype
ORDER BY total_sales DESC

Done.


mediatype,total_sales,percent_of_sales
MPEG audio file,42934.32,90.38
Protected AAC audio file,4115.43,8.66
Purchased AAC audio file,274.23,0.58
AAC audio file,153.45,0.32
Protected MPEG-4 video file,25.74,0.05


From looking at the sales by media type, we can see that MPEG audio files are the most popular, with a whopping 90.38% of total sales! This makes sense, as MPEG audio files are one of the most popular lossless compression audio file types, as they have been a standard since the 90s, and are supported by a wide variety of products and technologies. 

# Conclusion

From our SQL analysis of the Chinook digital record store, we can conclude that:

* Rock, Alternative Punk, and Metal are the three most popular genres of music from the tracks/albums sold at Chinook

* The U.S.A, Canada, and Brazil constitute the three countries with the most record sales, with the Czech Republic looking to being a promising source of revenue.

* Most of the purchases at Chinook are individual track purchases, and not full albums

* 2 of the 3 most popular artists (Iron Maiden and Metallica) by playlist appearances (# of times a track  by an artist is included on a playlist) are from the Metal genre

* The most popular artist by total sales is Jimi Hendrix, as 9 out of the top 10  highest selling tracks are from his album, 'Are You Experienced?'

* Of the top 10 tracks (by frequency) at Chinook that were never sold, most are video/audio files of popular TV Shows, indicating some need for change in decision making with regards to purchasing those type of tracks for Chinook  

* MPEG audio files are the most popular form of audio file that customers prefer to purchase their music

Hopefully, these insights will allow Chinook to take the necessary next steps in ensuring their record store can improve in their overall business strategy.
